In [ ]:
!pip uninstall tensorflow

In [ ]:
!pip install tensorflow==1.15.5

In [1]:
import numpy as np
import tensorflow as tf
import iris
import matplotlib.pyplot as plt
from neural_network_decision_tree import nn_decision_tree
import pandas as pd

In [2]:
tf.__version__

'1.15.5'

In [3]:
np.random.seed(1943)
tf.set_random_seed(1943)

In [168]:
dataset = pd.read_excel('/content/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx')

In [169]:
age = pd.get_dummies(dataset['AGE_PERCENTIL'],drop_first=True)
window = pd.get_dummies(dataset['WINDOW'],drop_first=True)
dataset.drop(['AGE_PERCENTIL','WINDOW'],axis=1,inplace=True)
dataset = pd.concat([dataset,age,window],axis=1)

In [177]:
x = dataset.iloc[:, 3:9] # use "Petal length" and "Petal width" only
y = dataset["ICU"]
d = x.shape[1]

In [144]:
num_cut = [1, 1]  # "Petal length" and "Petal width"
num_leaf = np.prod(np.array(num_cut) + 1)
num_class = 3

In [145]:
sess = tf.InteractiveSession()

/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [146]:
x_ph = tf.placeholder(tf.float32, [None, d])
y_ph = tf.placeholder(tf.float32, [None, num_class])

In [147]:
cut_points_list = [tf.Variable(tf.random_uniform([i])) for i in num_cut]
leaf_score = tf.Variable(tf.random_uniform([num_leaf, num_class]))

In [148]:
y_pred = nn_decision_tree(x_ph, cut_points_list, leaf_score, temperature=0.1)
loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(logits=y_pred, onehot_labels=y_ph))

In [149]:
opt = tf.train.AdamOptimizer(0.1)
train_step = opt.minimize(loss)

In [150]:
sess.run(tf.global_variables_initializer())

In [ ]:
for i in range(1000):
    _, loss_e = sess.run([train_step, loss], feed_dict={x_ph: x, y_ph: y})
    if i % 200 == 0:
        print(loss_e)
print('error rate %.2f' % (1 - np.mean(np.argmax(y_pred.eval(feed_dict={x_ph: x}), axis=1) == np.argmax(y, axis=1))))